NFL Big Data Bowl 2025 Data Ingestion 

In [10]:
import pyspark
import os 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType  

userDirectory = os.getcwd()
print(userDirectory)
appName = "NFLBigData2025"
master = "local" 
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.jars", f"{userDirectory}/nfl-big-data-bowl-2025/postgresql-42.6.2.jar") \
    .getOrCreate() 
print("Spark session started successfully with PostgreSQL driver")

/home/koza/Documents/NFLBigDataBowl2025/nfl-big-data-bowl-2025/NFLBigData2025_CMUTrack
Spark session started successfully with PostgreSQL driver


Games.csv Data Ingestion

In [11]:
from pyspark.sql.functions import col, isnan, when, count
games_schema = StructType([
    StructField("gameId", IntegerType(), True),
    StructField("season", IntegerType(), True),
    StructField("week", IntegerType(), True),
    StructField("gameDate", StringType(), True),  
    StructField("gameTimeEastern", StringType(), True),
    StructField("homeTeamAbbr", StringType(), True),
    StructField("visitorTeamAbbr", StringType(), True),
    StructField("homeFinalScore", IntegerType(), True),
    StructField("visitorFinalScore", IntegerType(), True)
])
games_df = spark.read.csv(f"{userDirectory}/Data/games.csv", header=True, schema=games_schema)

#games_df.printSchema()
#games_df.show(5)

#All Columns Contain Values No Need To Drop Anything
'''
total_rows = games_df.count()

missing_percentage = games_df.select([
    (count(when(col(c).isNull() | isnan(c), c)) / total_rows).alias(c) for c in games_df.columns
])

print("Missing Percentage per Column:")
missing_percentage.show()

columns_to_drop = [c for c in missing_percentage.columns if missing_percentage.collect()[0][c] > 0.75]
print(columns_to_drop)
games_df_cleaned = games_df.drop(*columns_to_drop)
games_df_cleaned = games_df_cleaned.fillna({
    "gameDate": "Unknown",
    "gameTimeEastern": "00:00:00",
    "homeTeamAbbr": "Unknown",
    "visitorTeamAbbr": "Unknown",
    "homeFinalScore": 0,
    "visitorFinalScore": 0
})
'''


24/12/28 18:39:38 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.


'\ntotal_rows = games_df.count()\n\nmissing_percentage = games_df.select([\n    (count(when(col(c).isNull() | isnan(c), c)) / total_rows).alias(c) for c in games_df.columns\n])\n\nprint("Missing Percentage per Column:")\nmissing_percentage.show()\n\ncolumns_to_drop = [c for c in missing_percentage.columns if missing_percentage.collect()[0][c] > 0.75]\nprint(columns_to_drop)\ngames_df_cleaned = games_df.drop(*columns_to_drop)\ngames_df_cleaned = games_df_cleaned.fillna({\n    "gameDate": "Unknown",\n    "gameTimeEastern": "00:00:00",\n    "homeTeamAbbr": "Unknown",\n    "visitorTeamAbbr": "Unknown",\n    "homeFinalScore": 0,\n    "visitorFinalScore": 0\n})\n'

play.csv Data Ingestion

In [12]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, BooleanType
from pyspark.sql.functions import col, isnan, when, count

plays_schema = StructType([
    StructField("gameId", IntegerType(), True),
    StructField("playId", IntegerType(), True),
    StructField("playDescription", StringType(), True),
    StructField("quarter", IntegerType(), True),
    StructField("down", IntegerType(), True),
    StructField("yardsToGo", IntegerType(), True),
    StructField("possessionTeam", StringType(), True),
    StructField("defensiveTeam", StringType(), True),
    StructField("yardlineSide", StringType(), True),
    StructField("yardlineNumber", IntegerType(), True),
    StructField("gameClock", StringType(), True),
    StructField("preSnapHomeScore", IntegerType(), True),
    StructField("preSnapVisitorScore", IntegerType(), True),
    StructField("playNullifiedByPenalty", BooleanType(), True),
    StructField("absoluteYardlineNumber", IntegerType(), True),
    StructField("preSnapHomeTeamWinProbability", FloatType(), True),
    StructField("preSnapVisitorTeamWinProbability", FloatType(), True),
    StructField("expectedPoints", FloatType(), True),
    StructField("offenseFormation", StringType(), True),
    StructField("receiverAlignment", StringType(), True),
    StructField("playClockAtSnap", StringType(), True),
    StructField("passResult", StringType(), True),
    StructField("passLength", FloatType(), True),
    StructField("targetX", FloatType(), True),
    StructField("targetY", FloatType(), True),
    StructField("playAction", BooleanType(), True),
    StructField("dropbackType", StringType(), True),
    StructField("dropbackDistance", FloatType(), True),
    StructField("passLocationType", StringType(), True),
    StructField("timeToThrow", FloatType(), True),
    StructField("timeInTackleBox", FloatType(), True),
    StructField("timeToSack", FloatType(), True),
    StructField("passTippedAtLine", BooleanType(), True),
    StructField("unblockedPressure", BooleanType(), True),
    StructField("qbSpike", BooleanType(), True),
    StructField("qbKneel", BooleanType(), True),
    StructField("qbSneak", BooleanType(), True),
    StructField("rushLocationType", StringType(), True),
    StructField("penaltyYards", IntegerType(), True),
    StructField("prePenaltyYardsGained", IntegerType(), True),
    StructField("yardsGained", IntegerType(), True),
    StructField("homeTeamWinProbabilityAdded", FloatType(), True),
    StructField("visitorTeamWinProbilityAdded", FloatType(), True),
    StructField("expectedPointsAdded", FloatType(), True),
    StructField("isDropback", BooleanType(), True),
    StructField("pff_runConceptPrimary", StringType(), True),
    StructField("pff_runConceptSecondary", StringType(), True),
    StructField("pff_runPassOption", BooleanType(), True),
    StructField("pff_passCoverage", StringType(), True),
    StructField("pff_manZone", StringType(), True)
])

plays_df = spark.read.csv(f"{userDirectory}/Data/plays.csv", header=True, schema=plays_schema)
#plays_df.printSchema()
#plays_df.show(5)


# Removing & Replacing Missing Values and Collumns with 75% missing data
plays_df = plays_df.withColumn(
    "pff_runPassOption",
    when(
        col("pff_runPassOption").cast("string").isin("true", "false"),
        col("pff_runPassOption").cast(BooleanType())
    ).otherwise(None)
)
total_rows = plays_df.count()

missing_percentage = plays_df.select([
    (
        count(when(col(c).isNull(), c)) / total_rows
        if plays_df.schema[c].dataType != FloatType()
        else count(when(col(c).isNull() | isnan(c), c)) / total_rows
    ).alias(c) for c in plays_df.columns
])
missing_percentage.show()

columns_to_drop = [
    c for c in missing_percentage.columns if missing_percentage.collect()[0][c] > 0.75
]
print(f"Number of columns dropped: {len(columns_to_drop)}")
print(f"Columns dropped: {columns_to_drop}")
plays_df_cleaned = plays_df.drop(*columns_to_drop)
remaining_columns = plays_df_cleaned.columns
default_values = {
    "gameId": -1,
    "playId": -1,
    "playDescription": "No Description",
    "quarter": 0,
    "down": 0,
    "yardsToGo": 0,
    "possessionTeam": "Unknown",
    "defensiveTeam": "Unknown",
    "yardlineSide": "N/A",
    "yardlineNumber": 0,
    "gameClock": "00:00",
    "preSnapHomeScore": 0,
    "preSnapVisitorScore": 0,
    "playNullifiedByPenalty": False,
    "absoluteYardlineNumber": 0,
    "preSnapHomeTeamWinProbability": 0.5,
    "preSnapVisitorTeamWinProbability": 0.5,
    "expectedPoints": 0.0,
    "offenseFormation": "Unknown",
    "receiverAlignment": "Unknown",
    "playClockAtSnap": "00:00",
    "passResult": "Unknown",
    "passLength": 0.0,
    "targetX": 0.0,
    "targetY": 0.0,
    "playAction": False,
    "dropbackType": "Unknown",
    "dropbackDistance": 0.0,
    "passLocationType": "Unknown",
    "timeToThrow": 0.0,
    "timeInTackleBox": 0.0,
    "timeToSack": 0.0,
    "passTippedAtLine": False,
    "unblockedPressure": False,
    "qbSpike": False,
    "qbKneel": False,
    "qbSneak": False,
    "rushLocationType": "Unknown",
    "penaltyYards": 0,
    "prePenaltyYardsGained": 0,
    "yardsGained": 0,
    "homeTeamWinProbabilityAdded": 0.0,
    "visitorTeamWinProbilityAdded": 0.0,
    "expectedPointsAdded": 0.0,
    "isDropback": False,
    "pff_runConceptPrimary": "Unknown",
    "pff_runConceptSecondary": "Unknown",
    "pff_runPassOption": False,
    "pff_passCoverage": "Unknown",
    "pff_manZone": "Unknown"
}
default_values_filtered = {k: v for k, v in default_values.items() if k in remaining_columns}
plays_df_cleaned = plays_df_cleaned.fillna(default_values_filtered)


24/12/28 18:39:38 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/12/28 18:39:39 INFO FileSourceStrategy: Pushed Filters: 
24/12/28 18:39:39 INFO FileSourceStrategy: Post-Scan Filters: 
24/12/28 18:39:39 INFO CodeGenerator: Code generated in 78.618134 ms
24/12/28 18:39:39 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 200.4 KiB, free 4.6 GiB)
24/12/28 18:39:39 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.7 KiB, free 4.6 GiB)
24/12/28 18:39:39 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 127.0.0.1:41575 (size: 34.7 KiB, free: 4.6 GiB)
24/12/28 18:39:39 INFO SparkContext: Created broadcast 0 from count at NativeMethodAccessorImpl.java:0
24/12/28 18:39:39 INFO FileSourceScanExec: Planning scan with bin packing, max size: 11086657 bytes, open cost is considered as scanning 4194304 bytes.
24/12/28 18:39:39 INFO DAGScheduler: Registering RDD 3 (count at NativeMethodAccessorImpl.

+------+------+---------------+-------+----+---------+--------------+-------------+------------+--------------+---------+----------------+-------------------+----------------------+----------------------+-----------------------------+--------------------------------+--------------+----------------+-----------------+---------------+-------------------+------------------+-------------------+-------------------+----------+------------+-------------------+----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-------+------------------+----------------+------------------+---------------------+-----------+---------------------------+----------------------------+-------------------+----------+---------------------+-----------------------+-----------------+----------------+-----------+
|gameId|playId|playDescription|quarter|down|yardsToGo|possessionTeam|defensiveTeam|yardlineSide|yardlineNumber|gameClock|preSnapHome

24/12/28 18:39:41 INFO BlockManagerInfo: Removed broadcast_3_piece0 on 127.0.0.1:41575 in memory (size: 34.7 KiB, free: 4.6 GiB)
24/12/28 18:39:41 INFO Executor: Finished task 0.0 in stage 6.0 (TID 4). 1968 bytes result sent to driver
24/12/28 18:39:41 INFO TaskSetManager: Finished task 0.0 in stage 6.0 (TID 4) in 369 ms on 10.0.0.36 (executor driver) (1/1)
24/12/28 18:39:41 INFO TaskSchedulerImpl: Removed TaskSet 6.0, whose tasks have all completed, from pool 
24/12/28 18:39:41 INFO DAGScheduler: ShuffleMapStage 6 (collect at /tmp/ipykernel_216402/637052616.py:82) finished in 0.374 s
24/12/28 18:39:41 INFO DAGScheduler: looking for newly runnable stages
24/12/28 18:39:41 INFO DAGScheduler: running: Set()
24/12/28 18:39:41 INFO DAGScheduler: waiting: Set()
24/12/28 18:39:41 INFO DAGScheduler: failed: Set()
24/12/28 18:39:41 INFO CodeGenerator: Code generated in 11.295337 ms
24/12/28 18:39:41 INFO SparkContext: Starting job: collect at /tmp/ipykernel_216402/637052616.py:82
24/12/28 18:3

Number of columns dropped: 5
Columns dropped: ['playNullifiedByPenalty', 'timeToSack', 'qbKneel', 'penaltyYards', 'pff_runPassOption']


24/12/28 18:39:42 INFO SparkContext: Starting job: collect at /tmp/ipykernel_216402/637052616.py:82
24/12/28 18:39:42 INFO DAGScheduler: Got job 51 (collect at /tmp/ipykernel_216402/637052616.py:82) with 1 output partitions
24/12/28 18:39:42 INFO DAGScheduler: Final stage: ResultStage 100 (collect at /tmp/ipykernel_216402/637052616.py:82)
24/12/28 18:39:42 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 99)
24/12/28 18:39:42 INFO DAGScheduler: Missing parents: List()
24/12/28 18:39:42 INFO DAGScheduler: Submitting ResultStage 100 (MapPartitionsRDD[112] at collect at /tmp/ipykernel_216402/637052616.py:82), which has no missing parents
24/12/28 18:39:42 INFO MemoryStore: Block broadcast_54 stored as values in memory (estimated size 62.9 KiB, free 4.6 GiB)
24/12/28 18:39:42 INFO MemoryStore: Block broadcast_54_piece0 stored as bytes in memory (estimated size 17.0 KiB, free 4.6 GiB)
24/12/28 18:39:42 INFO BlockManagerInfo: Added broadcast_54_piece0 in memory on 127.0.0.1:41

player_play.csv Data Ingestion

This data set has mostly blank or zero information, so most likley will not use at all 

In [13]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType
from pyspark.sql.functions import col, isnan, when, count

playerplay_schema = StructType([
    StructField("gameId", IntegerType(), True),
    StructField("playId", IntegerType(), True),
    StructField("nflId", IntegerType(), True),
    StructField("teamAbbr", StringType(), True),
    StructField("hadRushAttempt", IntegerType(), True),
    StructField("rushingYards", FloatType(), True),
    StructField("hadDropback", IntegerType(), True),
    StructField("passingYards", FloatType(), True),
    StructField("sackYardsAsOffense", FloatType(), True),
    StructField("hadPassReception", IntegerType(), True),
    StructField("receivingYards", FloatType(), True),
    StructField("wasTargettedReceiver", IntegerType(), True),
    StructField("yardageGainedAfterTheCatch", FloatType(), True),
    StructField("fumbles", IntegerType(), True),
    StructField("fumbleLost", IntegerType(), True),
    StructField("fumbleOutOfBounds", IntegerType(), True),
    StructField("assistedTackle", IntegerType(), True),
    StructField("forcedFumbleAsDefense", IntegerType(), True),
    StructField("halfSackYardsAsDefense", FloatType(), True),
    StructField("passDefensed", IntegerType(), True),
    StructField("quarterbackHit", IntegerType(), True),
    StructField("sackYardsAsDefense", FloatType(), True),
    StructField("safetyAsDefense", IntegerType(), True),
    StructField("soloTackle", IntegerType(), True),
    StructField("tackleAssist", IntegerType(), True),
    StructField("tackleForALoss", IntegerType(), True),
    StructField("tackleForALossYardage", FloatType(), True),
    StructField("hadInterception", IntegerType(), True),
    StructField("interceptionYards", FloatType(), True),
    StructField("fumbleRecoveries", IntegerType(), True),
    StructField("fumbleRecoveryYards", FloatType(), True),
    StructField("penaltyYards", IntegerType(), True),
    StructField("penaltyNames", StringType(), True),
    StructField("wasInitialPassRusher", IntegerType(), True),
    StructField("causedPressure", BooleanType(), True),
    StructField("timeToPressureAsPassRusher", FloatType(), True),
    StructField("getOffTimeAsPassRusher", FloatType(), True),
    StructField("inMotionAtBallSnap", BooleanType(), True),
    StructField("shiftSinceLineset", BooleanType(), True),
    StructField("motionSinceLineset", BooleanType(), True),
    StructField("wasRunningRoute", BooleanType(), True),
    StructField("routeRan", StringType(), True),
    StructField("blockedPlayerNFLId1", IntegerType(), True),
    StructField("blockedPlayerNFLId2", IntegerType(), True),
    StructField("blockedPlayerNFLId3", IntegerType(), True),
    StructField("pressureAllowedAsBlocker", IntegerType(), True),
    StructField("timeToPressureAllowedAsBlocker", FloatType(), True),
    StructField("pff_defensiveCoverageAssignment", StringType(), True),
    StructField("pff_primaryDefensiveCoverageMatchupNflId", IntegerType(), True),
    StructField("pff_secondaryDefensiveCoverageMatchupNflId", IntegerType(), True)
])

playerplay_data_df = spark.read.csv(f"{userDirectory}/Data/player_play.csv", header=True, schema=playerplay_schema)

total_rows = playerplay_data_df.count()

# Removing & Replacing Missing Values and Collumns with 75% missing data

missing_percentage = playerplay_data_df.select([
    (
        count(when(col(c).isNull(), c)) / total_rows
        if playerplay_data_df.schema[c].dataType != FloatType()
        else count(when(col(c).isNull() | isnan(c), c)) / total_rows
    ).alias(c) for c in playerplay_data_df.columns
])

missing_percentage.show()

columns_to_drop = [
    c for c in missing_percentage.columns if missing_percentage.collect()[0][c] > 0.5
]

print(f"Number of columns dropped: {len(columns_to_drop)}")
print(f"Columns dropped: {columns_to_drop}")

playerplay_data_df_cleaned = playerplay_data_df.drop(*columns_to_drop)

remaining_columns = playerplay_data_df_cleaned.columns

default_values = {
    "gameId": -1,
    "playId": -1,
    "nflId": -1,
    "teamAbbr": "Unknown",
    "hadRushAttempt": 0,
    "rushingYards": 0.0,
    "hadDropback": 0,
    "passingYards": 0.0,
    "sackYardsAsOffense": 0.0,
    "hadPassReception": 0,
    "receivingYards": 0.0,
    "wasTargettedReceiver": 0,
    "yardageGainedAfterTheCatch": 0.0,
    "fumbles": 0,
    "fumbleLost": 0,
    "fumbleOutOfBounds": 0,
    "assistedTackle": 0,
    "forcedFumbleAsDefense": 0,
    "halfSackYardsAsDefense": 0.0,
    "passDefensed": 0,
    "quarterbackHit": 0,
    "sackYardsAsDefense": 0.0,
    "safetyAsDefense": 0,
    "soloTackle": 0,
    "tackleAssist": 0,
    "tackleForALoss": 0,
    "tackleForALossYardage": 0.0,
    "hadInterception": 0,
    "interceptionYards": 0.0,
    "fumbleRecoveries": 0,
    "fumbleRecoveryYards": 0.0,
    "penaltyYards": 0,
    "penaltyNames": "No Penalty",
    "wasInitialPassRusher": 0,
    "causedPressure": False,
    "timeToPressureAsPassRusher": 0.0,
    "getOffTimeAsPassRusher": 0.0,
    "inMotionAtBallSnap": False,
    "shiftSinceLineset": False,
    "motionSinceLineset": False,
    "wasRunningRoute": False,
    "routeRan": "Unknown",
    "blockedPlayerNFLId1": -1,
    "blockedPlayerNFLId2": -1,
    "blockedPlayerNFLId3": -1,
    "pressureAllowedAsBlocker": 0,
    "timeToPressureAllowedAsBlocker": 0.0,
    "pff_defensiveCoverageAssignment": "Unknown",
    "pff_primaryDefensiveCoverageMatchupNflId": -1,
    "pff_secondaryDefensiveCoverageMatchupNflId": -1
}

default_values_filtered = {k: v for k, v in default_values.items() if k in remaining_columns}

playerplay_data_df_cleaned = playerplay_data_df_cleaned.fillna(default_values_filtered)

playerplay_data_df_cleaned.show(5)


24/12/28 18:39:42 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/12/28 18:39:42 INFO FileSourceStrategy: Pushed Filters: 
24/12/28 18:39:42 INFO FileSourceStrategy: Post-Scan Filters: 
24/12/28 18:39:42 INFO MemoryStore: Block broadcast_58 stored as values in memory (estimated size 200.4 KiB, free 4.6 GiB)
24/12/28 18:39:42 INFO MemoryStore: Block broadcast_58_piece0 stored as bytes in memory (estimated size 34.7 KiB, free 4.6 GiB)
24/12/28 18:39:42 INFO BlockManagerInfo: Added broadcast_58_piece0 in memory on 127.0.0.1:41575 (size: 34.7 KiB, free: 4.6 GiB)
24/12/28 18:39:42 INFO SparkContext: Created broadcast 58 from count at NativeMethodAccessorImpl.java:0
24/12/28 18:39:42 INFO FileSourceScanExec: Planning scan with bin packing, max size: 56728066 bytes, open cost is considered as scanning 4194304 bytes.
24/12/28 18:39:42 INFO DAGScheduler: Registering RDD 122 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 3
24/12/28 18:39:42 INFO DAGSchedul

+------+------+-----+--------+--------------+------------+-----------+------------+------------------+----------------+--------------+--------------------+--------------------------+-------+----------+-----------------+--------------+---------------------+----------------------+------------+--------------+------------------+---------------+----------+------------+--------------+---------------------+---------------+-----------------+----------------+-------------------+------------+------------+--------------------+--------------+--------------------------+----------------------+------------------+-----------------+------------------+---------------+--------+-------------------+-------------------+-------------------+------------------------+------------------------------+-------------------------------+----------------------------------------+------------------------------------------+
|gameId|playId|nflId|teamAbbr|hadRushAttempt|rushingYards|hadDropback|passingYards|sackYardsAsOffens

24/12/28 18:39:51 INFO BlockManagerInfo: Removed broadcast_63_piece0 on 127.0.0.1:41575 in memory (size: 17.9 KiB, free: 4.6 GiB)
24/12/28 18:39:59 INFO Executor: Finished task 0.0 in stage 113.0 (TID 59). 1968 bytes result sent to driver
24/12/28 18:39:59 INFO TaskSetManager: Finished task 0.0 in stage 113.0 (TID 59) in 7900 ms on 10.0.0.36 (executor driver) (1/1)
24/12/28 18:39:59 INFO TaskSchedulerImpl: Removed TaskSet 113.0, whose tasks have all completed, from pool 
24/12/28 18:39:59 INFO DAGScheduler: ShuffleMapStage 113 (collect at /tmp/ipykernel_216402/173790283.py:74) finished in 7.905 s
24/12/28 18:39:59 INFO DAGScheduler: looking for newly runnable stages
24/12/28 18:39:59 INFO DAGScheduler: running: Set()
24/12/28 18:39:59 INFO DAGScheduler: waiting: Set()
24/12/28 18:39:59 INFO DAGScheduler: failed: Set()
24/12/28 18:39:59 INFO CodeGenerator: Code generated in 9.404504 ms
24/12/28 18:39:59 INFO SparkContext: Starting job: collect at /tmp/ipykernel_216402/173790283.py:74
24

Number of columns dropped: 14
Columns dropped: ['wasInitialPassRusher', 'timeToPressureAsPassRusher', 'getOffTimeAsPassRusher', 'inMotionAtBallSnap', 'shiftSinceLineset', 'motionSinceLineset', 'wasRunningRoute', 'blockedPlayerNFLId1', 'blockedPlayerNFLId2', 'blockedPlayerNFLId3', 'pressureAllowedAsBlocker', 'timeToPressureAllowedAsBlocker', 'pff_primaryDefensiveCoverageMatchupNflId', 'pff_secondaryDefensiveCoverageMatchupNflId']
+----------+------+-----+--------+--------------+------------+-----------+------------+------------------+----------------+--------------+--------------------+--------------------------+-------+----------+-----------------+--------------+---------------------+----------------------+------------+--------------+------------------+---------------+----------+------------+--------------+---------------------+---------------+-----------------+----------------+-------------------+------------+------------+--------------+--------+-------------------------------+
|    g

24/12/28 18:40:00 INFO CodeGenerator: Code generated in 3.516486 ms
24/12/28 18:40:00 INFO Executor: Finished task 0.0 in stage 214.0 (TID 110). 2040 bytes result sent to driver
24/12/28 18:40:00 INFO TaskSetManager: Finished task 0.0 in stage 214.0 (TID 110) in 24 ms on 10.0.0.36 (executor driver) (1/1)
24/12/28 18:40:00 INFO TaskSchedulerImpl: Removed TaskSet 214.0, whose tasks have all completed, from pool 
24/12/28 18:40:00 INFO DAGScheduler: ResultStage 214 (showString at NativeMethodAccessorImpl.java:0) finished in 0.026 s
24/12/28 18:40:00 INFO DAGScheduler: Job 110 is finished. Cancelling potential speculative or zombie tasks for this job
24/12/28 18:40:00 INFO TaskSchedulerImpl: Killing all running tasks in stage 214: Stage finished
24/12/28 18:40:00 INFO DAGScheduler: Job 110 finished: showString at NativeMethodAccessorImpl.java:0, took 0.026756 s
24/12/28 18:40:00 INFO CodeGenerator: Code generated in 3.732508 ms


players.csv Data Ingestion

In [14]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, isnan, when, count

players_schema = StructType([
    StructField("nflId", IntegerType(), True),
    StructField("height", StringType(), True),
    StructField("weight", IntegerType(), True),
    StructField("birthDate", StringType(), True),
    StructField("collegeName", StringType(), True),
    StructField("position", StringType(), True),
    StructField("displayName", StringType(), True)
])

players_data_df = spark.read.csv(f"{userDirectory}/Data/players.csv", header=True, schema=players_schema)

total_rows = players_data_df.count()

# Removing & Replacing Missing Values and Collumns with 75% missing data

missing_percentage = players_data_df.select([
    (
        count(when(col(c).isNull(), c)) / total_rows
        if players_data_df.schema[c].dataType != StringType()
        else count(when(col(c).isNull() | (col(c) == ""), c)) / total_rows
    ).alias(c) for c in players_data_df.columns
])

missing_percentage.show()

columns_to_drop = [
    c for c in missing_percentage.columns if missing_percentage.collect()[0][c] > 0.75
]

print(f"Number of columns dropped: {len(columns_to_drop)}")
print(f"Columns dropped: {columns_to_drop}")

players_data_df_cleaned = players_data_df.drop(*columns_to_drop)

remaining_columns = players_data_df_cleaned.columns

default_values = {
    "nflId": -1,
    "height": "N/A",
    "weight": 0,
    "birthDate": "N/A",
    "collegeName": "Unknown",
    "position": "Unknown",
    "displayName": "Unknown"
}

default_values_filtered = {k: v for k, v in default_values.items() if k in remaining_columns}

players_data_df_cleaned = players_data_df_cleaned.fillna(default_values_filtered)

players_data_df_cleaned.show(5)


24/12/28 18:40:00 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/12/28 18:40:00 INFO FileSourceStrategy: Pushed Filters: 
24/12/28 18:40:00 INFO FileSourceStrategy: Post-Scan Filters: 
24/12/28 18:40:00 INFO MemoryStore: Block broadcast_118 stored as values in memory (estimated size 200.4 KiB, free 4.6 GiB)
24/12/28 18:40:00 INFO MemoryStore: Block broadcast_118_piece0 stored as bytes in memory (estimated size 34.7 KiB, free 4.6 GiB)
24/12/28 18:40:00 INFO BlockManagerInfo: Added broadcast_118_piece0 in memory on 127.0.0.1:41575 (size: 34.7 KiB, free: 4.6 GiB)
24/12/28 18:40:00 INFO SparkContext: Created broadcast 118 from count at NativeMethodAccessorImpl.java:0
24/12/28 18:40:00 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4299053 bytes, open cost is considered as scanning 4194304 bytes.
24/12/28 18:40:00 INFO DAGScheduler: Registering RDD 245 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 6
24/12/28 18:40:00 INFO DAGSche

+-----+------+------+---------+-----------+--------+-----------+
|nflId|height|weight|birthDate|collegeName|position|displayName|
+-----+------+------+---------+-----------+--------+-----------+
|  0.0|   0.0|   0.0|      0.0|        0.0|     0.0|        0.0|
+-----+------+------+---------+-----------+--------+-----------+

Number of columns dropped: 0
Columns dropped: []
+-----+------+------+----------+--------------+--------+--------------+
|nflId|height|weight| birthDate|   collegeName|position|   displayName|
+-----+------+------+----------+--------------+--------+--------------+
|25511|   6-4|   225|1977-08-03|      Michigan|      QB|     Tom Brady|
|29550|   6-4|   328|1982-01-22|      Arkansas|       T|  Jason Peters|
|29851|   6-2|   225|1983-12-02|    California|      QB| Aaron Rodgers|
|30842|   6-6|   267|1984-05-19|          UCLA|      TE|Marcedes Lewis|
|33084|   6-4|   217|1985-05-17|Boston College|      QB|     Matt Ryan|
+-----+------+------+----------+--------------+--

24/12/28 18:40:00 INFO SparkContext: Starting job: collect at /tmp/ipykernel_216402/162072258.py:31
24/12/28 18:40:00 INFO DAGScheduler: Got job 119 (collect at /tmp/ipykernel_216402/162072258.py:31) with 1 output partitions
24/12/28 18:40:00 INFO DAGScheduler: Final stage: ResultStage 229 (collect at /tmp/ipykernel_216402/162072258.py:31)
24/12/28 18:40:00 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 228)
24/12/28 18:40:00 INFO DAGScheduler: Missing parents: List()
24/12/28 18:40:00 INFO DAGScheduler: Submitting ResultStage 229 (MapPartitionsRDD[268] at collect at /tmp/ipykernel_216402/162072258.py:31), which has no missing parents
24/12/28 18:40:00 INFO MemoryStore: Block broadcast_129 stored as values in memory (estimated size 18.9 KiB, free 4.6 GiB)
24/12/28 18:40:00 INFO MemoryStore: Block broadcast_129_piece0 stored as bytes in memory (estimated size 7.5 KiB, free 4.6 GiB)
24/12/28 18:40:00 INFO BlockManagerInfo: Added broadcast_129_piece0 in memory on 127.0.0.

tracking_week_all.csv Data Ingestion

In [15]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, TimestampType
from pyspark.sql.functions import lit, col, isnan, when, count

tracking_schema = StructType([
    StructField("gameId", IntegerType(), True),
    StructField("playId", IntegerType(), True),
    StructField("nflId", IntegerType(), True),
    StructField("displayName", StringType(), True),
    StructField("frameId", IntegerType(), True),
    StructField("frameType", StringType(), True),
    StructField("time", TimestampType(), True),
    StructField("jerseyNumber", IntegerType(), True),
    StructField("club", StringType(), True),
    StructField("playDirection", StringType(), True),
    StructField("x", FloatType(), True),
    StructField("y", FloatType(), True),
    StructField("s", FloatType(), True),
    StructField("a", FloatType(), True),
    StructField("dis", FloatType(), True),
    StructField("o", FloatType(), True),
    StructField("dir", FloatType(), True),
    StructField("event", StringType(), True),
    StructField("week", IntegerType(), True)
])

tracking_df_with_week = None
# Removing & Replacing Missing Values and Collumns with 75% missing data
for i in range(1, 10):  
    tracking_file_path = f"{userDirectory}/Data/tracking_week_{i}.csv"
    df = spark.read.csv(tracking_file_path, header=True, schema=tracking_schema).withColumn("week", lit(i))
    tracking_df_with_week = df if tracking_df_with_week is None else tracking_df_with_week.union(df)

total_rows = tracking_df_with_week.count()

missing_percentage = tracking_df_with_week.select([
    (
        count(when(col(c).isNull(), c)) / total_rows
        if tracking_df_with_week.schema[c].dataType != FloatType()
        else count(when(col(c).isNull() | isnan(c), c)) / total_rows
    ).alias(c) for c in tracking_df_with_week.columns
])

missing_percentage.show()

columns_to_drop = [
    c for c in missing_percentage.columns if missing_percentage.collect()[0][c] > 0.75
]

print(f"Number of columns dropped: {len(columns_to_drop)}")
print(f"Columns dropped: {columns_to_drop}")

tracking_df_cleaned = tracking_df_with_week.drop(*columns_to_drop)

remaining_columns = tracking_df_cleaned.columns

default_values = {
    "gameId": -1,
    "playId": -1,
    "nflId": -1,
    "displayName": "Unknown",
    "frameId": 0,
    "frameType": "Unknown",
    "time": "1970-01-01 00:00:00",  
    "jerseyNumber": 0,
    "club": "Unknown",
    "playDirection": "Unknown",
    "x": 0.0,
    "y": 0.0,
    "s": 0.0,
    "a": 0.0,
    "dis": 0.0,
    "o": 0.0,
    "dir": 0.0,
    "event": "Unknown",
    "week": -1
}

default_values_filtered = {k: v for k, v in default_values.items() if k in remaining_columns}

tracking_df_cleaned = tracking_df_cleaned.fillna(default_values_filtered)

tracking_df_cleaned.show(5)


24/12/28 18:40:00 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/12/28 18:40:00 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/12/28 18:40:00 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/12/28 18:40:00 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/12/28 18:40:00 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/12/28 18:40:00 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/12/28 18:40:00 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/12/28 18:40:00 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/12/28 18:40:00 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/12/28 18:40:00 INFO FileSourceStrategy: Pushed Filters: 
24/12/28 18:40:00 INFO FileSourceStrategy: Post-Scan Filters: 
24/12/28 18:40:00 INFO FileSourceStrategy: Pushed Filters: 
24/12/28 18:40:00 INFO FileSourceS

+------+------+--------------------+-----------+-------+---------+----+--------------------+----+-------------+---+---+---+---+---+-------------------+-------------------+-----+----+
|gameId|playId|               nflId|displayName|frameId|frameType|time|        jerseyNumber|club|playDirection|  x|  y|  s|  a|dis|                  o|                dir|event|week|
+------+------+--------------------+-----------+-------+---------+----+--------------------+----+-------------+---+---+---+---+---+-------------------+-------------------+-----+----+
|   0.0|   0.0|0.043478260869565216|        0.0|    0.0|      0.0| 0.0|0.043478260869565216| 0.0|          0.0|0.0|0.0|0.0|0.0|0.0|0.04343996488770875|0.04343996488770875|  0.0| 0.0|
+------+------+--------------------+-----------+-------+---------+----+--------------------+----+-------------+---+---+---+---+---+-------------------+-------------------+-----+----+



24/12/28 18:41:54 INFO Executor: Finished task 0.0 in stage 243.0 (TID 256). 2416 bytes result sent to driver
24/12/28 18:41:54 INFO TaskSetManager: Starting task 1.0 in stage 243.0 (TID 257) (10.0.0.36, executor driver, partition 1, PROCESS_LOCAL, 9778 bytes) 
24/12/28 18:41:54 INFO Executor: Running task 1.0 in stage 243.0 (TID 257)
24/12/28 18:41:54 INFO TaskSetManager: Finished task 0.0 in stage 243.0 (TID 256) in 1640 ms on 10.0.0.36 (executor driver) (1/65)
24/12/28 18:41:54 INFO FileScanRDD: Reading File path: file:///home/koza/Documents/NFLBigDataBowl2025/nfl-big-data-bowl-2025/NFLBigData2025_CMUTrack/Data/tracking_week_1.csv, range: 134217728-268435456, partition values: [empty row]
24/12/28 18:41:55 INFO Executor: Finished task 1.0 in stage 243.0 (TID 257). 2416 bytes result sent to driver
24/12/28 18:41:55 INFO TaskSetManager: Starting task 2.0 in stage 243.0 (TID 258) (10.0.0.36, executor driver, partition 2, PROCESS_LOCAL, 9778 bytes) 
24/12/28 18:41:55 INFO Executor: Runn

Number of columns dropped: 0
Columns dropped: []
+----------+------+-----+--------------+-------+-----------+--------------------+------------+----+-------------+-----+-----+----+----+----+------+-----+--------------------+----+
|    gameId|playId|nflId|   displayName|frameId|  frameType|                time|jerseyNumber|club|playDirection|    x|    y|   s|   a| dis|     o|  dir|               event|week|
+----------+------+-----+--------------+-------+-----------+--------------------+------------+----+-------------+-----+-----+----+----+----+------+-----+--------------------+----+
|2022091200|    64|35459|Kareem Jackson|      1|BEFORE_SNAP|2022-09-13 00:16:...|          22| DEN|        right|51.06|28.55|0.72|0.37|0.07|246.17|68.34|huddle_break_offense|   1|
|2022091200|    64|35459|Kareem Jackson|      2|BEFORE_SNAP|2022-09-13 00:16:...|          22| DEN|        right|51.13|28.57|0.71|0.36|0.07|245.41|71.21|                  NA|   1|
|2022091200|    64|35459|Kareem Jackson|      3|BEF

24/12/28 18:43:32 INFO BlockManagerInfo: Removed broadcast_176_piece0 on 127.0.0.1:41575 in memory (size: 9.9 KiB, free: 4.6 GiB)
24/12/28 18:43:32 INFO BlockManagerInfo: Removed broadcast_173_piece0 on 127.0.0.1:41575 in memory (size: 9.9 KiB, free: 4.6 GiB)
24/12/28 18:43:32 INFO MemoryStore: Block broadcast_191_piece0 stored as bytes in memory (estimated size 34.7 KiB, free 4.6 GiB)
24/12/28 18:43:32 INFO BlockManagerInfo: Added broadcast_191_piece0 in memory on 127.0.0.1:41575 (size: 34.7 KiB, free: 4.6 GiB)
24/12/28 18:43:32 INFO SparkContext: Created broadcast 191 from showString at NativeMethodAccessorImpl.java:0
24/12/28 18:43:32 INFO BlockManagerInfo: Removed broadcast_185_piece0 on 127.0.0.1:41575 in memory (size: 9.9 KiB, free: 4.6 GiB)
24/12/28 18:43:32 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/12/28 18:43:32 INFO BlockManagerInfo: Removed broadcast_181_piece0 on 127.0.0.1:41575 

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.functions import when, col
from functools import reduce

df_folder = [games_df,plays_df_cleaned,players_data_df_cleaned,tracking_df_cleaned]


def classify_columns(df):
    numerical_cols = []
    categorical_cols = []
    binary_cols = []
    nominal_cols = []
    ordinal_cols = []

    for col_name in df.columns:
        col_type = dict(df.dtypes)[col_name]

        if col_type in ['int', 'double']:
            numerical_cols.append(col_name)
        else:
            categorical_cols.append(col_name)

    for col_name in categorical_cols:
        unique_values = df.select(col_name).distinct().count()

        if unique_values == 2:
            binary_cols.append(col_name)
        elif unique_values > 2:
            nominal_cols.append(col_name)

    for col_name in nominal_cols:
        if "rank" in col_name.lower() or "level" in col_name.lower():
            ordinal_cols.append(col_name)

    print("Classified Columns:")
    print(f"Numerical: {numerical_cols}")
    print(f"Categorical: {categorical_cols}")
    print(f"Binary: {binary_cols}")
    print(f"Nominal: {nominal_cols}")
    print(f"Ordinal: {ordinal_cols}")

    return {
        "numerical": numerical_cols,
        "categorical": categorical_cols,
        "binary": binary_cols,
        "nominal": nominal_cols,
        "ordinal": ordinal_cols
    }


def find_outliers(df, remove=False):
    numeric_columns = [column[0] for column in df.dtypes if column[1] in ['int', 'double']]
    for column in numeric_columns:
        Q1, Q3 = df.approxQuantile(column, [0.25, 0.75], relativeError=0)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        isOutlierCol = f'is_outlier_{column}'
        df = df.withColumn(
            isOutlierCol,
            when((col(column) < lower_bound) | (col(column) > upper_bound), 1).otherwise(0)
        )
    
    selected_columns = [column for column in df.columns if column.startswith("is_outlier")]
    df = df.withColumn('total_outliers', reduce(lambda x, y: x + y, [col(col_name) for col_name in selected_columns]))
    
    if remove:
        df = df.filter(col('total_outliers') == 0)
    
    df = df.drop(*selected_columns)  
    
    return df
def process_games_df(games_df):
    classified_columns = classify_columns(games_df)
    numerical_cols = classified_columns['numerical']

    if not numerical_cols:
        print("No numerical columns found for boxplot.")
        return classified_columns


    return classified_columns
def process_plays_df(plays_df_cleaned, plot, remove_outliers=False):
    print("Processing plays_df_cleaned...")
    classified_columns = classify_columns(plays_df_cleaned)

    print(f"Columns classified in plays_df_cleaned:")
    print(f"Numerical: {classified_columns['numerical']}")
    print(f"Categorical: {classified_columns['categorical']}")
    print(f"Binary: {classified_columns['binary']}")
    print(f"Nominal: {classified_columns['nominal']}")

    numerical_cols = classified_columns['numerical']
    if plot:
        for col in numerical_cols:
            try:
                plt.figure(figsize=(8, 6))
                sns.boxplot(x=plays_df_cleaned.select(col).toPandas()[col])
                plt.title(f"Boxplot of {col} (plays_df_cleaned)")
                plt.xlabel(col)
                plt.show()
            except Exception as e:
                print(f"Error creating boxplot for {col} in plays_df_cleaned: {e}")

    plays_df_with_outliers = find_outliers(plays_df_cleaned, remove=remove_outliers)
    
    unecessary_columns = ["playDescription", "possessionTeam", "defensiveTeam", "preSnapHomeTeamWinProbability", 
                          "preSnapVisitorTeamWinProbability", "expectedPoints", "yardsGained",
                         "homeTeamWinProbabilityAdded", "visitorTeamWinProbabilityAdded", "expectedPointsAdded"]
    
    plays_df_with_outliers = plays_df_with_outliers.drop(*unecessary_columns) 
    
    return plays_df_with_outliers



#process_games_df(games_df)
plays_df_cleaned_no_outliers = process_plays_df(plays_df_cleaned, plot=False, remove_outliers=False)
uneccesary_tracking_columns = ["displayName", "frameType", "jerseyNumber", "club"]


ModuleNotFoundError: No module named 'seaborn'

24/12/28 18:43:35 INFO CodeGenerator: Code generated in 2.417061 ms
24/12/28 18:43:36 INFO CodeGenerator: Code generated in 1.277381 ms
24/12/28 18:43:36 INFO FileScanRDD: Reading File path: file:///home/koza/Documents/NFLBigDataBowl2025/nfl-big-data-bowl-2025/NFLBigData2025_CMUTrack/Data/plays.csv, range: 0-6892353, partition values: [empty row]
24/12/28 18:43:36 INFO CodeGenerator: Code generated in 0.926711 ms
24/12/28 18:43:36 INFO Executor: Finished task 0.0 in stage 487.0 (TID 443). 488286 bytes result sent to driver
24/12/28 18:43:36 INFO TaskSetManager: Finished task 0.0 in stage 487.0 (TID 443) in 64 ms on 10.0.0.36 (executor driver) (1/1)
24/12/28 18:43:36 INFO TaskSchedulerImpl: Removed TaskSet 487.0, whose tasks have all completed, from pool 
24/12/28 18:43:36 INFO DAGScheduler: ResultStage 487 (approxQuantile at NativeMethodAccessorImpl.java:0) finished in 0.072 s
24/12/28 18:43:36 INFO DAGScheduler: Job 251 is finished. Cancelling potential speculative or zombie tasks for

24/12/28 18:44:25 INFO BlockManagerInfo: Removed broadcast_345_piece0 on 127.0.0.1:41575 in memory (size: 16.6 KiB, free: 4.6 GiB)
24/12/28 18:44:25 INFO BlockManagerInfo: Removed broadcast_343_piece0 on 127.0.0.1:41575 in memory (size: 16.6 KiB, free: 4.6 GiB)
24/12/28 18:44:25 INFO BlockManagerInfo: Removed broadcast_353_piece0 on 127.0.0.1:41575 in memory (size: 16.6 KiB, free: 4.6 GiB)
24/12/28 18:44:25 INFO BlockManagerInfo: Removed broadcast_349_piece0 on 127.0.0.1:41575 in memory (size: 16.6 KiB, free: 4.6 GiB)
24/12/28 18:44:25 INFO BlockManagerInfo: Removed broadcast_352_piece0 on 127.0.0.1:41575 in memory (size: 34.7 KiB, free: 4.6 GiB)
24/12/28 18:44:25 INFO BlockManagerInfo: Removed broadcast_342_piece0 on 127.0.0.1:41575 in memory (size: 34.7 KiB, free: 4.6 GiB)
24/12/28 18:44:25 INFO BlockManagerInfo: Removed broadcast_347_piece0 on 127.0.0.1:41575 in memory (size: 16.6 KiB, free: 4.6 GiB)
24/12/28 18:44:25 INFO BlockManagerInfo: Removed broadcast_344_piece0 on 127.0.0.1: